In [27]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e7/sample_submission.csv
/kaggle/input/playground-series-s4e7/train.csv
/kaggle/input/playground-series-s4e7/test.csv


In [28]:
#%pip install ydata-profiling

In [29]:
#from ydata_profiling import ProfileReport
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from scipy.stats import entropy
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# 1. Understanding the Problem and Objective:
Before diving into the data, I needed understand the problem I was trying to solve and the goals of this analysis. 

## About Data
For this Health Insurance data,this dataset is about an Insurance company that has provided Health Insurance to its customers. 

## Goal or Objective at  Hand
I need to build a model to predict whether the policyholders (customers) from past year will also be interested in Vehicle Insurance provided by the company.

In [30]:
# Importing libraries and Loading the Dataset:

import seaborn as sns
import matplotlib.pyplot as plt
import sklearn

train_dataset = pd.read_csv("/kaggle/input/playground-series-s4e7/train.csv")
test_dataset = pd.read_csv("/kaggle/input/playground-series-s4e7/test.csv")

In [31]:
# Taking a look at the structure of the Dataset
train_df = train_dataset.copy()

train_df.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,0,Male,21,1,35.0,0,1-2 Year,Yes,65101.0,124.0,187,0
1,1,Male,43,1,28.0,0,> 2 Years,Yes,58911.0,26.0,288,1
2,2,Female,25,1,14.0,1,< 1 Year,No,38043.0,152.0,254,0
3,3,Female,35,1,1.0,0,1-2 Year,Yes,2630.0,156.0,76,0
4,4,Female,36,1,15.0,1,1-2 Year,No,31951.0,152.0,294,0


In [32]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11504798 entries, 0 to 11504797
Data columns (total 12 columns):
 #   Column                Dtype  
---  ------                -----  
 0   id                    int64  
 1   Gender                object 
 2   Age                   int64  
 3   Driving_License       int64  
 4   Region_Code           float64
 5   Previously_Insured    int64  
 6   Vehicle_Age           object 
 7   Vehicle_Damage        object 
 8   Annual_Premium        float64
 9   Policy_Sales_Channel  float64
 10  Vintage               int64  
 11  Response              int64  
dtypes: float64(3), int64(6), object(3)
memory usage: 1.0+ GB


In [33]:
train_df.shape

(11504798, 12)

In [34]:
# Checking the Duplicate and Missing Values in the Dataset
train_df.duplicated().sum()
train_df.drop_duplicates(inplace=True)
train_df.shape

(11504798, 12)

In [35]:
missing_values = train_df.isnull().sum()
missing_values

id                      0
Gender                  0
Age                     0
Driving_License         0
Region_Code             0
Previously_Insured      0
Vehicle_Age             0
Vehicle_Damage          0
Annual_Premium          0
Policy_Sales_Channel    0
Vintage                 0
Response                0
dtype: int64

# 2. Exploratory Data Analysis(EDA)

In [36]:
#profile = ProfileReport(train_df, title="Profiling Report")

In [ ]:
#profile.to_notebook_iframe()

# 3. Data Preprocessing

In [37]:
train_df.head(10)

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,0,Male,21,1,35.0,0,1-2 Year,Yes,65101.0,124.0,187,0
1,1,Male,43,1,28.0,0,> 2 Years,Yes,58911.0,26.0,288,1
2,2,Female,25,1,14.0,1,< 1 Year,No,38043.0,152.0,254,0
3,3,Female,35,1,1.0,0,1-2 Year,Yes,2630.0,156.0,76,0
4,4,Female,36,1,15.0,1,1-2 Year,No,31951.0,152.0,294,0
5,5,Female,31,1,47.0,1,< 1 Year,No,28150.0,152.0,197,0
6,6,Male,23,1,45.0,1,< 1 Year,No,27128.0,152.0,190,0
7,7,Female,47,1,8.0,0,1-2 Year,Yes,40659.0,26.0,262,1
8,8,Female,26,1,28.0,1,< 1 Year,No,31639.0,152.0,36,0
9,9,Female,66,1,11.0,0,1-2 Year,Yes,2630.0,26.0,125,0


In [38]:
# Convert categorical variables
categorical_features = ['Gender', 'Vehicle_Age', 'Vehicle_Damage']
train_df = pd.get_dummies(train_df, columns=categorical_features, drop_first=True)

# Normalize/Standardize numerical features
numerical_features = ['Annual_Premium', 'Vintage']
scaler = StandardScaler()
train_df[numerical_features] = scaler.fit_transform(train_df[numerical_features])

train_df.head(10)

,id,Age,Driving_License,Region_Code,Previously_Insured,Annual_Premium,Policy_Sales_Channel,Vintage,Response,Gender_Male,Vehicle_Age_< 1 Year,Vehicle_Age_> 2 Years,Vehicle_Damage_Yes
0,0,21,1,35.0,0,2.105145,124.0,0.288852,0,True,False,False,True
1,1,43,1,28.0,0,1.728962,26.0,1.551675,1,True,False,True,True
2,2,25,1,14.0,1,0.460756,152.0,1.126566,0,False,True,False,False
3,3,35,1,1.0,0,-1.691389,156.0,-1.099003,0,False,False,False,True
4,4,36,1,15.0,1,0.090529,152.0,1.626694,0,False,False,False,False
5,5,31,1,47.0,1,-0.140468,152.0,0.413884,0,False,True,False,False
6,6,23,1,45.0,1,-0.202578,152.0,0.326362,0,True,True,False,False
7,7,47,1,8.0,0,0.619738,26.0,1.226592,1,False,False,False,True
8,8,26,1,28.0,1,0.071568,152.0,-1.599131,0,False,True,False,False
9,9,66,1,11.0,0,-1.691389,26.0,-0.486346,0,False,False,False,True


In [39]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11504798 entries, 0 to 11504797
Data columns (total 13 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   id                     int64  
 1   Age                    int64  
 2   Driving_License        int64  
 3   Region_Code            float64
 4   Previously_Insured     int64  
 5   Annual_Premium         float64
 6   Policy_Sales_Channel   float64
 7   Vintage                float64
 8   Response               int64  
 9   Gender_Male            bool   
 10  Vehicle_Age_< 1 Year   bool   
 11  Vehicle_Age_> 2 Years  bool   
 12  Vehicle_Damage_Yes     bool   
dtypes: bool(4), float64(4), int64(5)
memory usage: 833.9 MB


# 4. Feature Engineering

In [40]:
# Binning Age
train_df['Age_Bin'] = pd.cut(train_df['Age'], bins=[0, 30, 40, 50, 60, 70, 80], labels=[1, 2, 3, 4, 5, 6])

# Create Insurance History feature
train_df['Insurance_History'] = train_df['Previously_Insured'] * train_df['Vintage']

# Use quantile-based binning for normalized Vintage values
quantile_transformer = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='quantile', subsample=None)
train_df['Vintage_Bin'] = quantile_transformer.fit_transform(train_df[['Vintage']])

# Create a new feature combining vehicle damage status and vehicle age
train_df['Vehicle_Damage_Age'] = train_df['Vehicle_Damage_Yes'] * (
    train_df['Vehicle_Age_< 1 Year'] * 1 + 
    train_df['Vehicle_Age_> 2 Years'] * 2
)

# Calculate entropy for Policy_Sales_Channel
sales_channel_entropy = train_df['Policy_Sales_Channel'].value_counts(normalize=True)
train_df['Policy_Sales_Channel_Entropy'] = entropy(sales_channel_entropy)

In [41]:
# Check for missing values
train_df.isnull().sum()

id                                0
Age                               0
Driving_License                   0
Region_Code                       0
Previously_Insured                0
Annual_Premium                    0
Policy_Sales_Channel              0
Vintage                           0
Response                          0
Gender_Male                       0
Vehicle_Age_< 1 Year              0
Vehicle_Age_> 2 Years             0
Vehicle_Damage_Yes                0
Age_Bin                         601
Insurance_History                 0
Vintage_Bin                       0
Vehicle_Damage_Age                0
Policy_Sales_Channel_Entropy      0
dtype: int64

# 5. Prepare data for Modeling

In [42]:
# Convert bool variables
categorical_bool_features = ['Gender_Male', 'Vehicle_Age_< 1 Year', 'Vehicle_Age_> 2 Years', 'Vehicle_Damage_Yes']
train_df[categorical_bool_features] = train_df[categorical_bool_features].astype(int)


# Fill missing values for each column with the mode
for column in train_df.columns:
    mode_value = train_df[column].mode()[0]
    train_df.loc[:, column] = train_df[column].fillna(mode_value)

# Define the features and target variable
X = train_df.drop(['id', 'Age', 'Response'], axis=1)
y = train_df['Response']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [43]:
train_df.isnull().sum()

id                              0
Age                             0
Driving_License                 0
Region_Code                     0
Previously_Insured              0
Annual_Premium                  0
Policy_Sales_Channel            0
Vintage                         0
Response                        0
Gender_Male                     0
Vehicle_Age_< 1 Year            0
Vehicle_Age_> 2 Years           0
Vehicle_Damage_Yes              0
Age_Bin                         0
Insurance_History               0
Vintage_Bin                     0
Vehicle_Damage_Age              0
Policy_Sales_Channel_Entropy    0
dtype: int64

In [44]:
train_df.head(15)

,id,Age,Driving_License,Region_Code,Previously_Insured,Annual_Premium,Policy_Sales_Channel,Vintage,Response,Gender_Male,Vehicle_Age_< 1 Year,Vehicle_Age_> 2 Years,Vehicle_Damage_Yes,Age_Bin,Insurance_History,Vintage_Bin,Vehicle_Damage_Age,Policy_Sales_Channel_Entropy
0,0,21,1,35.0,0,2.105145,124.0,0.288852,0,1,0,0,1,1,0.000000,1.0,0,2.040305
1,1,43,1,28.0,0,1.728962,26.0,1.551675,1,1,0,1,1,3,0.000000,2.0,2,2.040305
2,2,25,1,14.0,1,0.460756,152.0,1.126566,0,0,1,0,0,1,1.126566,2.0,0,2.040305
3,3,35,1,1.0,0,-1.691389,156.0,-1.099003,0,0,0,0,1,2,-0.000000,0.0,0,2.040305
4,4,36,1,15.0,1,0.090529,152.0,1.626694,0,0,0,0,0,2,1.626694,2.0,0,2.040305
5,5,31,1,47.0,1,-0.140468,152.0,0.413884,0,0,1,0,0,2,0.413884,1.0,0,2.040305
6,6,23,1,45.0,1,-0.202578,152.0,0.326362,0,1,1,0,0,1,0.326362,1.0,0,2.040305
7,7,47,1,8.0,0,0.619738,26.0,1.226592,1,0,0,0,1,3,0.000000,2.0,0,2.040305
8,8,26,1,28.0,1,0.071568,152.0,-1.599131,0,0,1,0,0,1,-1.599131,0.0,0,2.040305
9,9,66,1,11.0,0,-1.691389,26.0,-0.486346,0,0,0,0,1,5,-0.000000,1.0,0,2.040305


In [45]:
'''
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
from sklearn.model_selection import KFold

# Initialize the models
models = {
    'Random Forest': RandomForestClassifier(random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42)
}

# Define the K-fold cross-validation object
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# Train and evaluate the models
model_scores = {}
for model_name, model in models.items():
    # Initialize the AUC-ROC scores
    auc_scores = []

    # Perform K-fold cross-validation
    for train_index, val_index in kfold.split(X_train):
        X_train_fold = X_train.iloc[train_index]
        y_train_fold = y_train.iloc[train_index]
        X_val_fold = X_train.iloc[val_index]
        y_val_fold = y_train.iloc[val_index]

        # Train the model on the current fold
        model.fit(X_train_fold, y_train_fold)

        # Make predictions on the validation set
        y_pred_proba_fold = model.predict_proba(X_val_fold)[:, 1]

        # Evaluate the model on the current fold
        auc = roc_auc_score(y_val_fold, y_pred_proba_fold)
        auc_scores.append(auc)

        # Print the AUC-ROC score for the current fold
        print(f'{model_name} Fold {train_index} AUC-ROC: {auc:.4f}')
        print('\n' + '='*60 + '\n')

    # Calculate the average AUC-ROC across all folds
    avg_auc = sum(auc_scores) / len(auc_scores)
    model_scores[model_name] = avg_auc
    print(f'{model_name} Average AUC-ROC: {avg_auc:.4f}')
    print('\n' + '='*60 + '\n')

# Compare the models and choose the best one
best_model_name = max(model_scores, key=model_scores.get)
best_model_avg_auc = model_scores[best_model_name]
print(f'Best Model: {best_model_name} with Average AUC-ROC: {best_model_avg_auc:.4f}')
'''

"\nfrom sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier\nfrom sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score\nfrom sklearn.model_selection import KFold\n\n# Initialize the models\nmodels = {\n    'Random Forest': RandomForestClassifier(random_state=42),\n    'Gradient Boosting': GradientBoostingClassifier(random_state=42)\n}\n\n# Define the K-fold cross-validation object\nkfold = KFold(n_splits=5, shuffle=True, random_state=42)\n\n# Train and evaluate the models\nmodel_scores = {}\nfor model_name, model in models.items():\n    # Initialize the AUC-ROC scores\n    auc_scores = []\n\n    # Perform K-fold cross-validation\n    for train_index, val_index in kfold.split(X_train):\n        X_train_fold = X_train.iloc[train_index]\n        y_train_fold = y_train.iloc[train_index]\n        X_val_fold = X_train.iloc[val_index]\n        y_val_fold = y_train.iloc[val_index]\n\n        # Train the model on the current fold\n

Code Cell Output:

Random Forest Fold [      1       4       5 ... 9203835 9203836 9203837] AUC-ROC: 0.8319

============================================================

Random Forest Fold [      0       1       2 ... 9203834 9203836 9203837] AUC-ROC: 0.8320

============================================================

Random Forest Fold [      0       1       2 ... 9203834 9203835 9203837] AUC-ROC: 0.8318

============================================================

Random Forest Fold [      0       2       3 ... 9203832 9203835 9203836] AUC-ROC: 0.8326

============================================================

Random Forest Fold [      0       1       2 ... 9203835 9203836 9203837] AUC-ROC: 0.8321

============================================================

Random Forest Average AUC-ROC: 0.8321

============================================================

Gradient Boosting Fold [      1       4       5 ... 9203835 9203836 9203837] AUC-ROC: 0.8625

============================================================

Gradient Boosting Fold [      0       1       2 ... 9203834 9203836 9203837] AUC-ROC: 0.8622

============================================================

Gradient Boosting Fold [      0       1       2 ... 9203834 9203835 9203837] AUC-ROC: 0.8621

============================================================

Gradient Boosting Fold [      0       2       3 ... 9203832 9203835 9203836] AUC-ROC: 0.8629

============================================================

Gradient Boosting Fold [      0       1       2 ... 9203835 9203836 9203837] AUC-ROC: 0.8616

============================================================

Gradient Boosting Average AUC-ROC: 0.8623

============================================================

In [ ]:
xgb_data = train_df.copy()

In [ ]:
xgb_data.columns = xgb_data.columns.str.replace('[', '').str.replace(']', '').str.replace('<', '').str.replace('>', '')

In [ ]:
feature_names = list(xgb_data.columns)
feature_names

In [ ]:
# Define the features and target variable for xgb
X1 = xgb_data.drop(['id', 'Age', 'Response'], axis=1)
y1 = xgb_data['Response']

# Split the data
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.2, random_state=42)

In [ ]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
from sklearn.model_selection import KFold

# Initialize the models
models = {
    'XGBoost': xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42, device='cuda', enable_categorical=True)
}

# Define the K-fold cross-validation object
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# Train and evaluate the models
model_scores = {}
for model_name, model in models.items():
    # Initialize the AUC-ROC scores
    auc_scores = []

    # Perform K-fold cross-validation
    for train_index, val_index in kfold.split(X_train1):
        X_train_fold = X_train1.iloc[train_index]
        y_train_fold = y_train1.iloc[train_index]
        X_val_fold = X_train1.iloc[val_index]
        y_val_fold = y_train1.iloc[val_index]

        # Train the model on the current fold
        model.fit(X_train_fold, y_train_fold)

        # Make predictions on the validation set
        y_pred_proba_fold = model.predict_proba(X_val_fold)[:, 1]

        # Evaluate the model on the current fold
        auc = roc_auc_score(y_val_fold, y_pred_proba_fold)
        auc_scores.append(auc)

        # Print the AUC-ROC score for the current fold
        print(f'{model_name} Fold {train_index} AUC-ROC: {auc:.4f}')
        print('\n' + '='*60 + '\n')

    # Calculate the average AUC-ROC across all folds
    avg_auc = sum(auc_scores) / len(auc_scores)
    model_scores[model_name] = avg_auc
    print(f'{model_name} Average AUC-ROC: {avg_auc:.4f}')
    print('\n' + '='*60 + '\n')

# Compare the models and choose the best one
best_model_name = max(model_scores, key=model_scores.get)
best_model_avg_auc = model_scores[best_model_name]
print(f'Best Model: {best_model_name} with Average AUC-ROC: {best_model_avg_auc:.4f}')